##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [2]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers.
Use 
* a Dense (=Fully-Connected) layer with 128 neuros and ReLu as activation function, [Doc](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)
* followed by a Dropout layout with a frequency of 20% [Doc](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout),
* and finally a Dense layer with 10 neurons.

In [23]:
x_train.shape

(60000, 28, 28)

In [24]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [42]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='adam'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

ValueError: Unknown activation function: 'adam'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [32]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               100480    
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


What is the Dropout layout?

The Dropout layer randomly sets input units to 0 with a frequency of rate at each step during training time, which helps prevent overfitting. Inputs not set to 0 are scaled up by 1/(1 - rate) such that the sum over all inputs is unchanged.

Note that the Dropout layer only applies when training is set to True such that no values are dropped during inference. When using model.fit, training will be appropriately set to True automatically, and in other contexts, you can set the kwarg explicitly to True when calling the layer.

(This is in contrast to setting trainable=False for a Dropout layer. trainable does not affect the layer's behavior, as Dropout does not have any variables/weights that can be frozen during training.)

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [33]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 1.0869646 , -0.18220249,  0.06994891,  0.0563377 ,  0.32451317,
         0.22350276, -0.28904626,  0.64175296, -0.03846464,  1.0602967 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [34]:
# TODO
tf.nn.softmax(predictions).numpy()

array([[0.19688089, 0.05533645, 0.07120644, 0.0702438 , 0.09184915,
        0.08302461, 0.04972899, 0.12613936, 0.06389043, 0.19169988]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example. [Doc](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy)

In [35]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [36]:
loss_fn(y_train[:1], predictions).numpy()

2.4886184

Compile the model with Adam optimizer, the loss function defined above and accuracy as the only metric.
* [Doc model.compile](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile)


In [37]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss.

Fit the model with x_train, y_train during 5 epochs.

[Doc](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit)

In [38]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: 0.3788 - accuracy: 0.8906
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3164 - accuracy: 0.9106
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3036 - accuracy: 0.9147
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2962 - accuracy: 0.9174
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2900 - accuracy: 0.9189


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [39]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.2873 - accuracy: 0.9219 - 820ms/epoch - 3ms/step


[0.2873341143131256, 0.9218999743461609]

The image classifier is now trained to ~98% accuracy on this dataset.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it.
Add a [Softmax layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Softmax):

In [16]:
probability_model = tf.keras.Sequential([
  model,
    tf.keras.layers.Softmax()
])

In [17]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.41204893e-07, 1.02368546e-07, 8.33520119e-07, 7.47082522e-04,
        1.69938768e-11, 1.47340234e-06, 7.14953975e-13, 9.99247074e-01,
        1.25342615e-06, 2.03687205e-06],
       [7.28275218e-09, 2.07794546e-05, 9.99967337e-01, 1.13726910e-05,
        1.75835464e-15, 5.48404159e-07, 1.82667614e-09, 1.54047903e-13,
        1.59721765e-08, 6.55215086e-14],
       [8.50210213e-08, 9.99196827e-01, 1.14500137e-04, 2.15133559e-06,
        4.47546290e-06, 8.88515387e-06, 4.48783203e-06, 5.80789463e-04,
        8.76843769e-05, 7.57777840e-08],
       [9.99975085e-01, 2.95086941e-11, 1.06138468e-05, 2.21928826e-06,
        1.04055928e-08, 3.63908202e-06, 7.58331271e-06, 3.11469279e-07,
        8.14466894e-09, 4.96753898e-07],
       [9.62518584e-07, 1.36078135e-08, 1.55631205e-05, 5.97419998e-08,
        9.99191225e-01, 2.40563463e-06, 5.67439110e-06, 1.47099854e-05,
        7.20325852e-06, 7.62320589e-04]], dtype=float32)>

In [28]:
y_test[:5]

array([7, 2, 1, 0, 4], dtype=uint8)